In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from ipyleaflet import (Map, Marker, basemaps, basemap_to_tiles,
                        FullScreenControl, LayersControl, LayerGroup,
                        WidgetControl, Circle
                       )
from ipywidgets import HTML, Layout, Dropdown, Output, Textarea, VBox, Label
from datamanager import load_wifi_at_location

def classify_wifi_strength(rssi):
    """
    Categories for WiFi RSSI Measurements
        Excellent: x >= -50 dbm
        Good: -50 dbm > x >= -60 dbm 
        Fair: -60 dbm > x >= -70 dbm
        Weak: -70 dbm x
        
    Parameters:
        rssi:integer
            The RSSI value
        
    Returns:
        The category of the WiFi signal strength
    """
    if rssi >= -50: return 'Excellent'
    elif rssi < -50 and rssi >= -60: return 'Good'
    elif rssi < -60 and rssi >= -70: return 'Fair'
    return 'Weak'

STRENGTH_COLOUR = {
    # Blue for Excellent
    'Excellent' : '#0A75F0',
    # Green for Good
    'Good' : '#0AF01D',
    # Yellow for Fair
    'Fair' : '#FFFB08',
    # Red for Weak
    'Weak' : '#FF0C08'
}

In [11]:
wifi_data = load_wifi_at_location('whitfeld')
wifi_data

,index,lat,lon,time,bssid,quality,rssi,essid
0,0,-31.976879,115.817472,20-09-12-15-48-16,A0:CF:5B:3D:5F:90,42/70,-68,Unifi
1,1,-31.976879,115.817472,20-09-12-15-48-16,A0:CF:5B:3D:5F:9F,36/70,-74,Unifi
2,2,-31.976879,115.817472,20-09-12-15-48-16,A0:CF:5B:3D:7F:90,36/70,-74,Unifi
3,3,-31.976879,115.817472,20-09-12-15-48-16,38:ED:18:D0:EF:6F,31/70,-79,Unifi
4,4,-31.976879,115.817472,20-09-12-15-48-16,40:CE:24:C4:DB:2F,27/70,-83,Unifi
...,...,...,...,...,...,...,...,...
176,87,-31.976034,115.818277,20-09-12-16-17-46,64:AE:0C:C4:EC:00,33/70,-77,Unifi
177,88,-31.975984,115.818277,20-09-12-16-17-56,A0:CF:5B:3D:82:70,30/70,-80,Unifi
178,89,-31.975984,115.818277,20-09-12-16-17-56,A0:CF:5B:3F:8A:C0,28/70,-82,Unifi
179,90,-31.975984,115.818277,20-09-12-16-17-56,A0:CF:5B:3D:82:7F,27/70,-83,Unifi


In [22]:
test = wifi_data.loc[wifi_data['bssid'] == 'A0:CF:5B:3D:7F:D0']

# for index, row in test.iterrows():
#     print(row['lat'], row['lon'], row['rssi'])
test

list(wifi_data['bssid'].unique())

['A0:CF:5B:3D:5F:90',
 'A0:CF:5B:3D:5F:9F',
 'A0:CF:5B:3D:7F:90',
 '38:ED:18:D0:EF:6F',
 '40:CE:24:C4:DB:2F',
 '38:ED:18:D0:EF:60',
 '40:CE:24:C4:DB:20',
 'A0:CF:5B:3E:16:B0',
 '40:CE:24:85:C4:00',
 'A0:CF:5B:3F:69:30',
 '40:CE:24:85:C4:0F',
 'A0:CF:5B:3F:87:E0',
 'A0:CF:5B:3E:16:BF',
 'A0:CF:5B:3F:69:3F',
 '70:DF:2F:9C:47:4F',
 'A0:CF:5B:3F:87:EF',
 'A0:CF:5B:3D:7F:D0',
 'A0:CF:5B:3D:83:70',
 'A0:CF:5B:3D:7F:DF',
 '5C:50:15:E7:EA:30',
 '10:BD:18:F0:C8:A0',
 '10:BD:18:F0:C8:AF',
 'A0:CF:5B:3D:83:7F',
 'A0:CF:5B:3F:88:E0',
 'A0:CF:5B:3F:88:EF',
 'A0:CF:5B:3D:82:70',
 'A0:CF:5B:0E:33:D0',
 'A0:CF:5B:0E:33:DF',
 'A0:CF:5B:3F:8A:C0',
 'A0:CF:5B:3D:82:7F',
 '64:AE:0C:C4:EC:00']

In [33]:
class WiFiMap:
    
    RADIUS_MODIFIER = 0.5
    CIRCLE_OPACITY = 1.0
    CIRCLE_WEIGHT = 2
    
    def __init__(self, wifi_df, length):
        """
        Initialises the WiFiMap class
        
        Parameters:
            wifi_df: pandas.dataframe
                The Dataframe with the WiFi measurements and locations
        """
        self.wifi_df = wifi_df
        self.current_df = wifi_df
        self.length = length
        
        # self.marker_layer = LayerGroup(name='markers')
        self.best_rssi_layer = LayerGroup(name='RSSI Values')
        
        self.map = Map(
            basemap = basemaps.Esri.WorldImagery,
            center = self.get_average_location(),
            zoom = 100
        )
        
        # self.map.add_layer(self.marker_layer)
        self.map.add_layer(self.best_rssi_layer)
        # self.map.add_layer(self.line_layer)
        self.map.add_control(FullScreenControl())
        self.map.add_control(LayersControl(position='topright'))
        self.create_dropdown()
        
    def create_dropdown(self):
        self.dropdown = Dropdown(
            options = ['All Access Points'] + list(wifi_data['bssid'].unique()),
            layout = Layout(width='auto')
        )
        
        def on_ap_select(change):
            new_ap = self.dropdown.value
            if new_ap == 'All Access Points':
                self.current_df = self.wifi_df
            else:
                self.current_df = wifi_data.loc[wifi_data['bssid'] == new_ap]
            self.update_map()
            
        self.dropdown.observe(on_ap_select, names='value')
        
    def update_map(self):
        self.best_rssi_layer.clear_layers()
        best_strengths = self.current_df.groupby(['lat', 'lon'])['rssi'].max()
        
        for loc, rssi in best_strengths.items():
            colour = STRENGTH_COLOUR[classify_wifi_strength(rssi)]
            circle = Circle(
                location = loc,
                color = colour,
                fill_color = colour,
                radius = int(self.length*WiFiMap.RADIUS_MODIFIER),
                opacity = WiFiMap.CIRCLE_OPACITY,
                weight = WiFiMap.CIRCLE_WEIGHT
            )
            self.best_rssi_layer.add_layer(circle)
    
    def display_map(self):
        self.update_map()
        return VBox([self.map, self.dropdown])
        
    def get_average_location(self, bssid_filter=False):
        data_df = self.current_df
        return(data_df['lat'].mean(), data_df['lon'].mean())

In [34]:
wifi_map = WiFiMap(wifi_data, 5)
wifi_map.display_map()